In [1]:
from os import chdir

chdir("..")

from app.core.config import settings

# LLM

In [2]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_ollama.chat_models import ChatOllama

chat_model = ChatOllama(
    base_url=settings.OLLAMA_BASE_URL,
    model=settings.OLLAMA_CHAT_MODEL,
    temperature=0,
    # seed=1234,
    # num_ctx=500,
)

embeddings = OllamaEmbeddings(base_url=settings.OLLAMA_BASE_URL, model=settings.OLLAMA_EMBEDDINGS_MODEL)

# Vector DB

In [3]:
from langchain_chroma import Chroma

PERSIST_DIRECTORY = "./data/vector_db/chroma/"
vectordb = Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embeddings)

In [4]:
retriever = vectordb.as_retriever()

# RAG

In [5]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [30]:
from pydantic import BaseModel, field_validator

class Navigation(BaseModel):
    id: int
    reasoning: str

In [31]:
chat_model_structured = chat_model.with_structured_output(Navigation, method="json_schema")

In [32]:
chat_model_structured.invoke("Your ID is 1, reasoning is 'test case'")

Navigation(id=1, reasoning='test case')

In [39]:
from langchain_core.prompts import PromptTemplate


RAG_PROMPT = """
You are an navigation assistant. You are given a list of navigation options with their ID.
Your task is to identify the option that is most similar to the user {query}
Query: {query}
Context: {context}
Schema:
{{

    "id": ID,
    "reasoning": <Reasoning for selecting this ID
}}
Output:
"""

template = PromptTemplate.from_template(RAG_PROMPT)

rag_chain = template | chat_model_structured 

In [40]:
def retrieve(state: State):
    retrieved_docs = vectordb.similarity_search(state["question"])
    return {"context": retrieved_docs}

In [41]:
def generate(state: State):
    context = ""
    for doc in state["context"]:
        context += f"{doc.metadata['id']} - {doc.page_content} | "

    print(f"Context: {context}")
    
    response = rag_chain.invoke({"query": state["question"], "context": context})
    return {"answer": response}

In [42]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [49]:
QUERY = "user wants to login"

In [50]:
response = graph.invoke({"question": QUERY})
print(response["answer"])

Context: 1 - Directs the user to the login page to authenticate and access the application. | 16 - Directs to the page for adding a new user to the system. | 9 - Takes the user to the user list page to view and manage user profiles. | 17 - Navigates to the user preview page to review user details before saving. | 
id=16 reasoning='Directs to the page for adding a new user to the system.'
